# Catalogs in IBIS/ISGRI

Technically, ISGRI spectral and lightcurve extraction uses a `isgri_srcl_res.fits` catalog result.
Technically, the only way to get the `isgri_srcl_res.fits` is by running the mosaic. Of course, it is possible to fudge any other catalog in the necessary format (this is done in [MMODA](https://www.astro.unige.ch/mmoda/) when a catalog is generated from json provided by frontend or API), but this fudging is not naturally supported by `OSA` and should be done with care.

The reason why the mosaic is need in standard user workflow in OSA is because the spectral extraction quality depends strongly on the catalog - the sky model used for deconstructing the shadowgram. With a wrong catalog, unaccounted source counts will be misattributed to other sources (distorting them) and to background. If the counts are attributed to background, source fluxes are not disrupted. But due to wide PSFs, sources in some positions overlap substantially.

The good catalog is also needed to produce image - so the process is iterative (image -> catalog -> image -> catalog ... -> catalog -> spectra/lc ). This is the major source of complexity for coded mask analysis (and not only, this applies for every instrument with complex PSF, e.g. Fermi/LAT, or SKA).

As explained in OSA doc and https://volodymyrss.github.io/integral-ibis-isgri-le-response/catalogs.html (TBC), to get accurate reconstruction of a source-of-interest (SOI), it is necessary to take into account a catalog with sources down to roughly 3 times source-of-interest count rate (keeping in mind various nuances, e.g. in PCFoV the psf overlap approaches 100%, and sources of comparable flux affect each other strongly).

General reference catalog is the best global catalog for INTEGRAL, but it is not time-dependent. 
Using only GENREFCAT one often has too many sources in a catalog, and would miss some sources which are only detectable in short outbursts.

So, it is always advisable to produce an catalog for the observation of interest. INTEGRAL General Reference Catalog provides such a general model, with source fluxes indicated.
But, this catalog is static, so it is not really possible to derive sufficiently accurate catalog for given period, considering that over long time, we know that the sources are variable.

In the simplest case, when running spectral extraction directly in one ScW pipeline but it will only rely on the general reference catalog and imaging of this one ScW. This works at minimum, but this is not recommended even at the typical observer level.


We'd need a time-dependent catalog with all sources accounted on all timescales from ScW at least to whole mission. To be clear, to account for all sources, it is necessary to have intermediate timescales (orbit, year, etc).
But it should be clear that this is an enormous task, that's why so much effort goes in production of INTEGRAL catalog papers, and it was never done in an "theoretically ideal" way.
The best approximations of this approach are used in different pipelines, used by us or other people.



So a spectral/lc extraction has to rely on imaging made roughly for the same observation, adding computation.
It also means that typical lightcurves are produced for a variety of effectively equivalent but slightly different catalogs, creating some duplicated lightcurves with slightly different provenance.

With the data collection available now, we can provide the "recommended good" catalogs for spectral extraction at any time.

This could be an extension of REFCAT but with indication of validity of the measurement. If put in a single catalog, it would have millions of rows. This is best handled in a relational DB.

Ultimately, it is better to see the goal in designing a procedure for producing a catalog for given observation. The catalog can be built by querying the collection of available mosaics and providing the current best guess. The collection would have some 10k rows, and can be queried from an available store.

In [15]:
# adopted from  https://renkulab.io/projects/vladimir.savchenko/integral-ref-cat/files/blob/catsearch.ipynb


import numpy as np
import astropy
import astropy.coordinates

cat = astropy.io.fits.open("http://isdcarc.unige.ch/arc/rev_3/cat/hec/gnrl_refr_cat_0043.fits")
cat_table = cat[1].data
cat_coord = astropy.coordinates.SkyCoord(cat[1].data['RA_OBJ'], cat[1].data['DEC_OBJ'], unit='deg')

len(cat_table)

2410